In [1]:
%matplotlib inline
import os
import sys
from datetime import datetime
import logging
logging.basicConfig(format='%(asctime)s %(message)s', stream=sys.stdout)

import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
class Log():
    def __init__(self):
        pass
    
    def print(self, msg):
        print(f'{datetime.now().strftime("%H:%M:%S")} {msg}')
        
L = Log()

# Functions for reading data

In [3]:
def read_dataset(model: str, task: str, split: str,  basepath='/scratch/fl1092/ml_protein_data'):
    if model == 'transformer' and split == 'test': split='test_fold_holdout'
    path_to_file = f'{basepath}/{model}/{task}/{task}_{split}.p'
    data = np.load(path_to_file, allow_pickle=True)
    
    return data

def dict_2_arr(data_dict, labels, avgr=lambda x: np.mean(x, axis=0)):
    
    emb_shape = list(data_dict.values())[0].shape
    number_of_embeddings = len(data_dict) 

    X = np.zeros((number_of_embeddings, emb_shape[-1]))
    y = np.zeros(number_of_embeddings)
    
    i = 0

    # iter over sorted keys in labels to ensure proteins
    # from different models are indexed the same
    keys = list(labels.keys())
    keys.sort()
    for key in keys :
        if key in data_dict:
            X[i] = avgr(data_dict[key])
            y[i] = labels[key]
            i += 1
        
    return X, y

def ensemble_append_mean_reps(dicts, labels, LEN, average=True, normalize=True):
    # if average set to False, output 2d arrays for each sequence without averaging
    if LEN == -1:
        LEN = float('inf')
        
    keys = set(dicts[0].keys())
    for i in range(1, len(dicts)):
        keys = keys.intersection(set(dicts[i].keys()))
    
    # combine two dictionary into one by concatenating
    new_dict = dict()
    for key in keys:
        seqs = []
        for d in dicts:
            
            # 1d or 2d
            if average:
                seq = np.mean(d[key], axis=0)
            else:
                seq = d[key]
                if seq.shape[0] < LEN:
                    seq = np.concatenate([seq, np.zeros((LEN-seq.shape[0], seq.shape[1]))], axis=0)
                elif seq.shape[0] > LEN:
                    seq = seq[:LEN, ]
                
            # normalize or not
            if normalize:
                seq = preprocessing.normalize(seq.reshape(1, -1), norm='l2')

            seqs.append(seq)
        combined_seqs = np.concatenate(seqs, axis=-1)
        
        if average:
            combined_seqs[0]
        
        new_dict[key] = combined_seqs
    
    emb_size = combined_seqs.shape
    
    if average:
        X = np.zeros((len(new_dict), emb_size[1]))
    else:
        d1, d2 = emb_size
        X = np.zeros((len(new_dict), d1, d2))
    y = np.zeros(len(new_dict))
    
    i = 0
    for key in new_dict:
        X[i] = new_dict[key]
        y[i] = labels[key]
        i += 1
        
    print('concatenated embedding size: ', X.shape)
        
    return X, y

In [4]:
class DataSet(T.utils.data.Dataset):
    ### a Map-style dataset ###
    
    def __init__(self, task, split, average=True, normalize=True, conv=True, seq_len=-1):
        self.models = ['elmo','unirep', 'transformer'] 
        self.splits = ['train', 'test']
        self.x_data = {}
        self.y_data = {}
        self.LEN = seq_len
        self.split = split # the split of the current data
        
        # for split in self.splits:
        # load y
        y_data = read_dataset('label', task, self.split)

        # load x
        to_append = []
        for model in self.models:
            data = read_dataset(model, task, self.split)
            to_append.append(data)

        # concatnate
        X_app, self.y_data[split] = ensemble_append_mean_reps(to_append, y_data, self.LEN, average, normalize)
        if conv:
            # add channel information if convolutional neural net
            X_app = X_app.reshape((X_app.shape[0], 1, X_app.shape[1], X_app.shape[2]))
        print('Shape after appending: ', X_app.shape)
        
        self.x_data[self.split] = T.tensor(X_app, dtype=T.float32)
            
    def get_split(self, split):
        assert(split in self.splits)
        self.split = split
        
    def lenLongestSeq(self, task):
        # find the length of the longest sequence
        # TODO: trim and pad everyting to 95% interval
        LEN = -1
        for split in self.splits:
            for model in self.models:
                data = read_dataset(model, task, split)
                
                length = -1
                for key, embedding in data.items():
                    length = max(length, embedding.shape[0])
                
                print(f"Longest sequence in {model} {split}: {length}")
                    
                LEN = max(LEN, length)
        
        return LEN
    
    def lenRangeSeq(self, task, p=0.05):
        # return the uper p-th percentile
        lens = []
        
        for split in self.splits:
            for model in self.models:
                data = read_dataset(model, task, split)
                
                for key, embedding in data.items():
                    lens.append(embedding.shape[0])
                    
        return 
        
    def __len__(self):
        return len(self.x_data[self.split])

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        
        x = self.x_data[self.split][idx, ]
        y = self.y_data[self.split][idx]
        
        return x, y

# Bulding blocks for autoencoder

In [5]:
### encoder and decoder with two convolutional layers ###

class ConvEncoder(nn.Module):
    def __init__(self, c=10, latent_dims=512):
        super(ConvEncoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=c, kernel_size=4, stride=4, padding=1) # out: c x 14 x 14
        self.conv2 = nn.Conv2d(in_channels=c, out_channels=c*2, kernel_size=4, stride=4, padding=1) # out: c x 7 x 7
        self.fc = nn.Linear(in_features=c*2*16*231, out_features=latent_dims)
            
    def forward(self, x):
        #print(list(x.size()))
        x = F.relu(self.conv1(x))
        #print(list(x.size()))
        x = F.relu(self.conv2(x))
        #print(list(x.size()))
        x = x.view(x.size(0), -1) # flatten batch of multi-channel feature maps to a batch of feature vectors
        #print(list(x.size()))
        x = self.fc(x)
        #print(list(x.size()))
        return x

class ConvDecoder(nn.Module):
    def __init__(self, c=10, latent_dims=512):
        super(ConvDecoder, self).__init__()
        self.c = c
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2*16*231)
        self.conv2 = nn.ConvTranspose2d(in_channels=c*2, out_channels=c, kernel_size=4, stride=4, padding=1)
        self.conv1 = nn.ConvTranspose2d(in_channels=c, out_channels=1, kernel_size=4, stride=4, padding=1)
            
    def forward(self, x):
        #print(list(x.size()))
        x = self.fc(x)
        #print(list(x.size()))
        # unflatten batch of feature vectors to a batch of multi-channel feature maps
        x = x.view(x.size(0), self.c*2, 16, 231)
        #print(list(x.size()))
        x = F.relu(self.conv2(x, output_size=[128, 10, 64, 923]))
        #print(list(x.size()))
        # last layer before output is tanh, since the images are normalized and 0-centered
        x = T.tanh(self.conv1(x, output_size=[128, 1, 256, 3692]))
        #print(list(x.size()))
        return x

In [6]:
### encoder and decoder with fully connected layers only ###

class Encoder(nn.Module):
    def __init__(self, input_shape):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=input_shape, out_features=600)
        self.fc2 = nn.Linear(in_features=1024, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=128)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x

class Decoder(nn.Module):
    def __init__(self, input_shape):
        super(Decoder, self).__init__()
        #self.fc1 = nn.Linear(in_features=128, out_features=128)
        #self.fc2 = nn.Linear(in_features=128, out_features=1024)
        self.fc3 = nn.Linear(in_features=600, out_features=input_shape)
            
    def forward(self, x):
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [7]:
class Autoencoder(nn.Module):
    def __init__(self, input_shape, conv=False, latent_dims=512):
        super(Autoencoder, self).__init__()
        if conv:
            L.print('Convolutional neural network')
            self.encoder = ConvEncoder(latent_dims=latent_dims)
            self.decoder = ConvDecoder(latent_dims=latent_dims)
        else:
            L.print('Fully connected neural network')
            self.encoder = Encoder(input_shape)
            self.decoder = Decoder(input_shape)
    
    def forward(self, x):
        latent = self.encoder(x)
        x_recon = self.decoder(latent)
        return x_recon

In [8]:
### loss functions ###

def caemeLoss(outputs, batch_features, latent, d1, d2, l1=1, l2=1, l3=1):
    mse = nn.MSELoss()
    o1 = outputs[:, 0:d1]
    o2 = outputs[:, d1:d1+d2]
    f1 = batch_features[:, 0:d1]
    f2 = batch_features[:, d1:d1+d2] 
    
    return l1*mse(o1, f1) + l2*mse(o2, f2)

def mseLoss(outputs, batch_features, latent, d1, d2, l1=1, l2=1, l3=1):
    # mean-squared error loss
    mse = nn.MSELoss() 
    
    return mse(outputs, batch_features)

# Training pipeline

In [9]:
# def metaEmbedding(data, model, device):
#     X_latent = []
#     Y = []

#     for i in range(len(data)):
#         x, y = data[i]
#         X_latent.append(model.encoder(x.to(device)).cpu().detach().numpy())
#         Y.append(y)
        
#     return X_latent, Y

In [10]:
def metaEmbedding(data, model, device, conv=False):
    X_latent = []
    Y = []

    for i in range(len(data)):
        x, y = data[i]
        # print(x.shape)
        
        if conv:
            x = x.reshape(1, x.size()[0], x.size()[1], x.size()[2])
        else:
            x = x.reshape(1, x.size()[0])
        # print(x.shape)
            
        x = x.to(device)
        X_latent.append(model.encoder(x).cpu().detach().numpy()[0])
        Y.append(y)
        
    return X_latent, Y

In [11]:
### pipeline for training meta embedding and classifying ###

def autoencoderPipeline(task, conv=True, normalize=True, latent_dimension=128, seq_len=-1,
                        num_epochs=30, learning_rate=1e-3, batch_size=128, loss_function=mseLoss):
    # if seq len is set to -1, then don't trim when doing cnn
    
    device = T.device('cuda')
    if conv:
        average=False
    else:
        average=True
        
    if conv:
        # TODO: remove this
        assert(normalize==False)
    
    L.print('Loading data ...')
    train_data = DataSet(task, 'train', average, normalize, conv, seq_len)
    test_data = DataSet(task, 'test', average, normalize, conv, seq_len)
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    model = Autoencoder(3692, conv = conv, latent_dims=latent_dimension).to(device)
    optimizer = T.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-7)
    
    L.print('Training autoencoder...')
    for epoch in tqdm(range(num_epochs)):
        loss = 0
        for batch_features, _ in train_dataloader:
            
            batch_features = batch_features.to(device) # load data to GPU
            optimizer.zero_grad() # reset the gradients back to zero
            
            outputs = model(batch_features) # compute reconstructions
            latent = model.encoder(batch_features) # compute latent meta embedding
            
            train_loss = loss_function(outputs, batch_features, latent, 1024, 1900) # compute training loss
            train_loss.backward() # compute accumulated gradients
            optimizer.step() # perform parameter update based on current gradients
            loss += train_loss.item() # add the mini-batch training loss to epoch loss

        loss = loss / len(train_dataloader) # compute the epoch training loss

        if epoch %3 == 0:
            L.print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, num_epochs, loss))
            
    L.print('Training classifier ...')
    
    test_x_latent, test_y = metaEmbedding(test_data, model, device, conv)
    train_x_latent, train_y = metaEmbedding(train_data, model, device, conv)
    
    clf = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', multi_class='auto'))
    clf.fit(train_x_latent, train_y)
    
    score = clf.score(test_x_latent, test_y)
    
    L.print(f"Score on test set: {score}")

# Experiments

In [12]:
%%time
autoencoderPipeline('remote_homology', conv=False, normalize=True, latent_dimension=512, seq_len=-1)

21:27:41 Loading data ...
concatenated embedding size:  (12305, 3692)
Shape after appending:  (12305, 3692)
concatenated embedding size:  (718, 3692)
Shape after appending:  (718, 3692)
21:28:38 Fully connected neural network
21:28:53 Training autoencoder...


21:28:53 epoch : 1/30, loss = 0.000577
21:28:54 epoch : 4/30, loss = 0.000494
21:28:55 epoch : 7/30, loss = 0.000479
21:28:56 epoch : 10/30, loss = 0.000474
21:28:57 epoch : 13/30, loss = 0.000471
21:28:57 epoch : 16/30, loss = 0.000468
21:28:58 epoch : 19/30, loss = 0.000468
21:28:59 epoch : 22/30, loss = 0.000467
21:29:00 epoch : 25/30, loss = 0.000467
21:29:01 epoch : 28/30, loss = 0.000466

21:29:02 Training classifier ...
22:12:44 Score on test set: 0.2562674094707521
CPU times: user 43min 46s, sys: 51.4 s, total: 44min 38s
Wall time: 45min 3s


autoencoderPipeline('remote_homology', conv=False, normalize=True, latent_dimension=512, seq_len=-1)

21:27:41 Loading data ...
concatenated embedding size:  (12305, 3692)
Shape after appending:  (12305, 3692)
concatenated embedding size:  (718, 3692)
Shape after appending:  (718, 3692)
21:28:38 Fully connected neural network
21:28:53 Training autoencoder...
100% 30/30 [00:08<00:00, 3.33it/s]
21:28:53 epoch : 1/30, loss = 0.000577
21:28:54 epoch : 4/30, loss = 0.000494
21:28:55 epoch : 7/30, loss = 0.000479
21:28:56 epoch : 10/30, loss = 0.000474
21:28:57 epoch : 13/30, loss = 0.000471
21:28:57 epoch : 16/30, loss = 0.000468
21:28:58 epoch : 19/30, loss = 0.000468
21:28:59 epoch : 22/30, loss = 0.000467
21:29:00 epoch : 25/30, loss = 0.000467
21:29:01 epoch : 28/30, loss = 0.000466

21:29:02 Training classifier ...
22:12:44 Score on test set: 0.2562674094707521
CPU times: user 43min 46s, sys: 51.4 s, total: 44min 38s
Wall time: 45min 3s

In [12]:
%%time
autoencoderPipeline('remote_homology', conv=True, normalize=False, latent_dimension=512, seq_len=256)

22:18:00 Loading data ...
concatenated embedding size:  (12305, 256, 3692)
Shape after appending:  (12305, 1, 256, 3692)
concatenated embedding size:  (718, 256, 3692)
Shape after appending:  (718, 1, 256, 3692)
22:21:47 Convolutional neural network
22:22:05 Training autoencoder...


22:22:39 epoch : 1/30, loss = 0.125733
22:24:25 epoch : 4/30, loss = 0.091975
22:26:14 epoch : 7/30, loss = 0.086808
22:28:02 epoch : 10/30, loss = 0.084883
22:29:52 epoch : 13/30, loss = 0.083792
22:31:41 epoch : 16/30, loss = 0.083202
22:33:30 epoch : 19/30, loss = 0.083029
22:35:19 epoch : 22/30, loss = 0.082932
22:37:10 epoch : 25/30, loss = 0.082626
22:38:58 epoch : 28/30, loss = 0.082623

22:40:12 Training classifier ...
23:29:10 Score on test set: 0.25348189415041783
CPU times: user 1h 1min 29s, sys: 17min 5s, total: 1h 18min 35s
Wall time: 1h 11min 14s


22:18:00 Loading data ...
concatenated embedding size:  (12305, 256, 3692)
Shape after appending:  (12305, 1, 256, 3692)
concatenated embedding size:  (718, 256, 3692)
Shape after appending:  (718, 1, 256, 3692)
22:21:47 Convolutional neural network
22:22:05 Training autoencoder...
100% 30/30 [18:06<00:00, 36.23s/it]
22:22:39 epoch : 1/30, loss = 0.125733
22:24:25 epoch : 4/30, loss = 0.091975
22:26:14 epoch : 7/30, loss = 0.086808
22:28:02 epoch : 10/30, loss = 0.084883
22:29:52 epoch : 13/30, loss = 0.083792
22:31:41 epoch : 16/30, loss = 0.083202
22:33:30 epoch : 19/30, loss = 0.083029
22:35:19 epoch : 22/30, loss = 0.082932
22:37:10 epoch : 25/30, loss = 0.082626
22:38:58 epoch : 28/30, loss = 0.082623

22:40:12 Training classifier ...
23:29:10 Score on test set: 0.25348189415041783
CPU times: user 1h 1min 29s, sys: 17min 5s, total: 1h 18min 35s
Wall time: 1h 11min 14s

In [ ]:
%%time
autoencoderPipeline('remote_homology', conv=True, normalize=False, latent_dimension=512, seq_len=512)

00:01:15 Loading data ...


In [ ]:
%%time
autoencoderPipeline('remote_homology', conv=False, normalize=True, latent_dimension=3692, seq_len=-1)

00:08:37 Loading data ...
concatenated embedding size:  (12305, 3692)
Shape after appending:  (12305, 3692)
concatenated embedding size:  (718, 3692)
Shape after appending:  (718, 3692)
00:09:36 Fully connected neural network
00:10:07 Training autoencoder...


00:10:08 epoch : 1/30, loss = 0.000580
00:10:09 epoch : 4/30, loss = 0.000506
00:10:09 epoch : 7/30, loss = 0.000488
00:10:10 epoch : 10/30, loss = 0.000481
00:10:11 epoch : 13/30, loss = 0.000477
00:10:12 epoch : 16/30, loss = 0.000475
00:10:13 epoch : 19/30, loss = 0.000474
00:10:14 epoch : 22/30, loss = 0.000474
00:10:15 epoch : 25/30, loss = 0.000473
00:10:16 epoch : 28/30, loss = 0.000473

00:10:16 Training classifier ...


In [ ]:
%%time
autoencoderPipeline('remote_homology', conv=True, normalize=False, latent_dimension=3692, seq_len=1024)

In [10]:
task = 'remote_homology'
batch_size = 128

device = T.device('cuda')

print('Loading data ...')

train_data = DataSet(task, 'train', average=False, normalize=False)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

Loading data ...
Shape after appending:  (12305, 1, 256, 3692)


In [11]:
len(train_data)

12305

In [12]:
learning_rate = 1e-3
num_epochs = 30
model = Autoencoder(2924, conv=True).to(device)
optimizer = T.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-7)
loss_function = mseLoss

print('Training autoencoder...')
for epoch in tqdm(range(num_epochs)):
    loss = 0
    for batch_features, _ in train_dataloader:

        batch_features = batch_features.to(device) # load data to GPU
        optimizer.zero_grad() # reset the gradients back to zero

        outputs = model(batch_features) # compute reconstructions
        latent = model.encoder(batch_features) # compute latent meta embedding
        # print(list(outputs.size()), list(batch_features.size()))
        
        train_loss = loss_function(outputs, batch_features, latent, 1024, 1900) # compute training loss
        train_loss.backward() # compute accumulated gradients
        optimizer.step() # perform parameter update based on current gradients
        loss += train_loss.item() # add the mini-batch training loss to epoch loss

    loss = loss / len(train_dataloader) # compute the epoch training loss

    if epoch %3 == 0:
        print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, num_epochs, loss))

Training autoencoder...


epoch : 1/30, loss = 0.131212
epoch : 4/30, loss = 0.094607
epoch : 7/30, loss = 0.087955
epoch : 10/30, loss = 0.085494
epoch : 13/30, loss = 0.084361
epoch : 16/30, loss = 0.083872
epoch : 19/30, loss = 0.083340
epoch : 22/30, loss = 0.083148
epoch : 25/30, loss = 0.082889
epoch : 28/30, loss = 0.082713



In [13]:
test_data = DataSet(task, 'test', average=False, normalize=False)
test_data.get_split('test')

Shape after appending:  (718, 1, 256, 3692)


In [14]:
len(test_data)

718

In [15]:
def metaEmbedding(data, model):
    X_latent = []
    Y = []

    for i in range(len(data)):
        x, y = data[i]
        x = x.reshape(1, x.size()[0], x.size()[1], x.size()[2]).to(device)
        X_latent.append(model.encoder(x).cpu().detach().numpy()[0])
        Y.append(y)
        
    return X_latent, Y

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [17]:
%%time
test_x_latent, test_y = metaEmbedding(test_data, model)
train_x_latent, train_y = metaEmbedding(train_data, model)

CPU times: user 19.3 s, sys: 4.66 s, total: 24 s
Wall time: 24 s


In [18]:
%%time
clf = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', multi_class='auto'))
clf.fit(train_x_latent, train_y)

CPU times: user 48min 37s, sys: 1min 1s, total: 49min 38s
Wall time: 49min 45s


In [19]:
score = clf.score(test_x_latent, test_y)

In [20]:
score # 0.2479108635097493

0.25348189415041783

In [ ]:
learning_rate = 1e-3
num_epochs = 30
model = Autoencoder(2924, conv=True, latent_dims=2048).to(device)
optimizer = T.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-7)
loss_function = mseLoss

print('Training autoencoder...')
for epoch in tqdm(range(num_epochs)):
    loss = 0
    for batch_features, _ in train_dataloader:

        batch_features = batch_features.to(device) # load data to GPU
        optimizer.zero_grad() # reset the gradients back to zero

        outputs = model(batch_features) # compute reconstructions
        latent = model.encoder(batch_features) # compute latent meta embedding
        # print(list(outputs.size()), list(batch_features.size()))
        
        train_loss = loss_function(outputs, batch_features, latent, 1024, 1900) # compute training loss
        train_loss.backward() # compute accumulated gradients
        optimizer.step() # perform parameter update based on current gradients
        loss += train_loss.item() # add the mini-batch training loss to epoch loss

    loss = loss / len(train_dataloader) # compute the epoch training loss

    if epoch %3 == 0:
        print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, num_epochs, loss))

In [27]:
score

0.2479108635097493

In [28]:
clf = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear'))
clf.fit(train_x_latent, train_y)

/home/fl1092/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False))])

In [29]:
score = clf.score(test_x_latent, test_y)

In [30]:
score

0.2479108635097493

# Experiments

In [13]:
%%time
autoencoderPipeline('remote_homology', loss_function=mseLoss)

Loading data ...
Training autoencoder...


epoch : 1/30, loss = 0.000525
epoch : 4/30, loss = 0.000464
epoch : 7/30, loss = 0.000450
epoch : 10/30, loss = 0.000443
epoch : 13/30, loss = 0.000440
epoch : 16/30, loss = 0.000439
epoch : 19/30, loss = 0.000438
epoch : 22/30, loss = 0.000437
epoch : 25/30, loss = 0.000437
epoch : 28/30, loss = 0.000437

Training classifier ...
Score on test set: 0.25487465181058494
CPU times: user 3min 50s, sys: 35.7 s, total: 4min 26s
Wall time: 4min 25s


In [14]:
%%time
autoencoderPipeline('remote_homology', loss_function=caemeLoss)

Loading data ...
Training autoencoder...


epoch : 1/30, loss = 0.001178
epoch : 4/30, loss = 0.001067
epoch : 7/30, loss = 0.001046
epoch : 10/30, loss = 0.001031
epoch : 13/30, loss = 0.001025
epoch : 16/30, loss = 0.001021
epoch : 19/30, loss = 0.001018
epoch : 22/30, loss = 0.001015
epoch : 25/30, loss = 0.001014
epoch : 28/30, loss = 0.001013

Training classifier ...
Score on test set: 0.2646239554317549
CPU times: user 3min 49s, sys: 36 s, total: 4min 25s
Wall time: 4min 24s
